In [14]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

ProgrammingError: Oracle Client library has already been initialized

In [15]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [16]:
# qualifications

# select distinct DTN, WIRE_CENTER_ID
# from LQSMO.LU_CIRCUITS_O;


# 10/7/22 - update query to count unique LU where servicetype like XGS
# 10/19/22 - update details query to include NDSJOBID - per Scott request
query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            ), MG AS (
            SELECT  distinct
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
                    -- add multigig polygons below --
                    AND UNI_POLYGONID IN ('1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494','1189253', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306',-- MPLSMNBE 
                                          '1180028', '1178694', '1179569','1180333',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924','1179922', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237','1180468', -- CRYSMNCR
                                          '1180315','1189074','1189058', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507','1180076','1180079','1180492','1180497','1180486',  --TCSNAZMA
                                          '1180091','1179913','1189283',  --TCSNAZMA
                                          '1179641','1179914', --TCSNAZCR
                                          '1179840','1180355', --STTLWA04
                                          '1180086','1180111','1180327','1180436', --STTLWASU
                                          '1179576','1179672','1179971','1179977','1179979','1179981','1179987', --CLSPCOMA
                                          '1179988','1179989','1179994','1180005','1180006','1180041','1180123', --CLSPCOMA
                                          '1189130', --MLWKOR17
                                          '1179352','1180002','1189088', --MAGNUTNM
                                          '1179858','1180015','1180098','1180392','1180399', --SPKNWA01
                                          '1180027', --BRCTMNBC
                                          '1180491','1180509', --PHNXAZNO
                                          '1180488','1180496', --TCSNAZRN
                                          '1189200' --DNVRCOCL                          
                         )
                    
            ), QMG AS (
            SELECT  DISTINCT
                    QD.LUID
                    ,QD.PRIMARY_WIRE_CENTER_ID
                    ,QD.STATE
                    ,QD.UNI_SERVICETYPE
                    ,QD.UNI_POLYGONID
                    ,MG.UNI_POLYGONID AS MG_ENABLED_POLYGON

            FROM    QD
                    LEFT JOIN MG
                        ON QD.UNI_POLYGONID = MG.UNI_POLYGONID            
            )
            
            SELECT  0 as s
                    ,STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,UNI_SERVICETYPE
                    ,count(distinct LUID) as cnt
                    ,case when UNI_SERVICETYPE like '%XGS%' then count(distinct LUID) end as servicetype_xgs_cnt
                    ,count(case when MG_ENABLED_POLYGON is not null then 1 end) as multigig_cnt
                    
            FROM    QMG
                    
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,UNI_SERVICETYPE

            UNION ALL
            
            SELECT  1
                    ,'ALL'
                    ,'GRAND TOTAL'
                    ,'ALL'
                    ,'ALL'
                    ,count(distinct LUID)
                    ,count(distinct case when UNI_SERVICETYPE like '%XGS%' then LUID end)
                    ,count(case when MG_ENABLED_POLYGON is not null then 1 end)
                    
            FROM    QMG
            
            ORDER BY 
                    s
                    ,STATE

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    ,O.NDSJOBID
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  *
                    
            FROM    QD

                                """

# query3 = """

# select *
# FROM    LQSMO.LU_QUAL_O O

# """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
df_ora[df_ora['PRIMARY_WIRE_CENTER_ID']=='ARVDCOMA']

,S,STATE,PRIMARY_WIRE_CENTER_ID,UNI_POLYGONID,UNI_SERVICETYPE,CNT,SERVICETYPE_XGS_CNT,MULTIGIG_CNT
58,0,CO,ARVDCOMA,1180386,XGS,77,77.0,0
64,0,CO,ARVDCOMA,1178832,XGS,196,196.0,196


In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df_ora.rename(columns = {"CNT": "LU_QUAL_O_CNT",
                         "UNI_POLYGONID": "POLYGON_ID"}, inplace = True)
df_ora.sort_values(by = ['S','STATE']).head() # contains state/wc/polygon info

,S,STATE,PRIMARY_WIRE_CENTER_ID,POLYGON_ID,UNI_SERVICETYPE,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT,MULTIGIG_CNT
0,0,AZ,TCSNAZMA,1180497,XGS,560,560.0,560
1,0,AZ,TCSNAZCR,1179641,XGS,176,176.0,176
2,0,AZ,TCSNAZMA,1179704,XGS,550,550.0,550
3,0,AZ,TCSNAZRN,1180496,XGS,269,269.0,269
4,0,AZ,TCSNAZMA,1180486,XGS,545,545.0,545


In [19]:
# check data
# df_ora.loc[df_ora["MULTIGIG_CNT"]>0,:]

In [20]:
ST_df = pd.read_excel(r'Qualifications ST/Data.xlsx',sheet_name = 'Export')

ST_Details_df = ST_df.copy() # create copy to export

ST_df["Polygon ID - Phase Else Site"] = ST_df["Polygon ID - Phase Else Site"].fillna("None")
ST_df = ST_df.groupby(by = ["State","PRIMARY_WIRE_CENTER_ID","Polygon ID - Phase Else Site"], as_index = False)["# of LUs - Phase Else Site"].sum()
ST_df = ST_df.rename(columns = {'State': 'STATE',
                                'Polygon ID - Phase Else Site': 'POLYGON_ID'})
#ST_df

C:\Users\AD21236\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [21]:
# polygon level info starts here

df_ora_state = df_ora.copy() # use this for state summary with unitype cnt
df_ora = df_ora.groupby(['S','STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID'], as_index = False).sum()
Q_df = ST_df.merge(df_ora[["STATE","PRIMARY_WIRE_CENTER_ID", "POLYGON_ID","LU_QUAL_O_CNT","MULTIGIG_CNT"]], how = "outer", on = ["PRIMARY_WIRE_CENTER_ID","STATE","POLYGON_ID"]).fillna(0)
Q_df[["# of LUs - Phase Else Site", "LU_QUAL_O_CNT"]] = Q_df[["# of LUs - Phase Else Site", "LU_QUAL_O_CNT"]].astype(int)
Q_df.loc[Q_df["PRIMARY_WIRE_CENTER_ID"] == "GRAND TOTAL", ["# of LUs - Phase Else Site"]] = Q_df["# of LUs - Phase Else Site"].sum()
Q_df = Q_df[['STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID','# of LUs - Phase Else Site','LU_QUAL_O_CNT','MULTIGIG_CNT']]
Q_poly_df = Q_df.copy() # polygon level summary ready for export

In [22]:
# wc level information start here
Q_df = Q_df.groupby(by = ['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum() # wc level info ready for export
Q_df['ORDER'] = np.where(Q_df['PRIMARY_WIRE_CENTER_ID'] == 'GRAND TOTAL', 1, 0)
Q_df = Q_df.sort_values(by = 'ORDER')
Q_df = Q_df.drop(['ORDER'], axis = 1)

In [23]:
# state summary begins here
df_ora_state = df_ora_state.fillna(0)
Q_State1 = df_ora_state.groupby(['STATE'], as_index = False).sum()
Q_State2 = Q_df.groupby(['STATE'], as_index = False).sum()
Q_State = Q_State1[['STATE','LU_QUAL_O_CNT','SERVICETYPE_XGS_CNT','MULTIGIG_CNT']].merge(Q_State2[['STATE','# of LUs - Phase Else Site']], on = ['STATE'], how = 'outer')
Q_State['STATE'] = Q_State['STATE'].replace(['ALL'], 'GRAND TOTAL')
Q_State['ORDER'] = np.where(Q_State['STATE'] == 'GRAND TOTAL', 1, 0)
Q_State = Q_State.sort_values(by = 'ORDER')
Q_State = Q_State.drop(['ORDER'], axis = 1)
Q_State = Q_State.fillna(0)
Q_State = Q_State.reindex(columns = ['STATE', '# of LUs - Phase Else Site', 'LU_QUAL_O_CNT', 'SERVICETYPE_XGS_CNT', 'MULTIGIG_CNT'])
Q_State

,STATE,# of LUs - Phase Else Site,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT,MULTIGIG_CNT
1,AZ,17354,16987,16987.0,14316
2,CO,9913,9276,9276.0,4920
3,FL,28521,27539,27535.0,0
4,MN,10549,10508,10508.0,9711
5,NV,135,94,94.0,0
6,OR,426,349,349.0,349
7,UT,4890,4060,4060.0,659
8,WA,2530,2450,2450.0,2080
0,GRAND TOTAL,74318,71263,71259.0,32035


In [24]:
import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_LUQ_SITETRACKER_Summary.xlsx"


with pd.ExcelWriter(excelfilename) as writer:
    Q_poly_df.to_excel(writer, sheet_name='PolygonID_Summary', index = False)
    Q_df.to_excel(writer, sheet_name = "WC_Summary", index=False)
    Q_State.to_excel(writer, sheet_name='State_Summary', index = False)
    ST_Details_df.to_excel(writer, sheet_name='ST_Details', index = False)
    df_ora_details.to_excel(writer, sheet_name='LUQUAL_Details', index = False)
    

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()

In [25]:
conn.close()